In [ ]:
import pandas as pd
import numpy as np
import pymysql
import datetime
from nltk.tokenize import RegexpTokenizer
import gensim
import codecs
from sklearn.decomposition import PCA
from matplotlib import pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
import joblib

pd.set_option('display.max_columns', None)


def get_sql_data(sql):
    conn = pymysql.connect(host='localhost', user='root', password='1qaz!QAZ', database='twitter')
    sqlcmd = sql
    data = pd.read_sql(sqlcmd, conn)
    conn.close()
    return data


def ab(df):
    return ' '.join(df.values)


def all_posts_oneday(data):
    posts = data.groupby(['ticker', 'created_time'])['text'].apply(ab).reset_index()
    posts['retweets'] = data.groupby(['ticker', 'created_time'])['retweets'].apply(sum).reset_index().iloc[:, -1]
    posts['replies'] = data.groupby(['ticker', 'created_time'])['replies'].apply(sum).reset_index().iloc[:, -1]
    posts['likes'] = data.groupby(['ticker', 'created_time'])['likes'].apply(sum).reset_index().iloc[:, -1]
    posts['is_retweet'] = data.groupby(['ticker', 'created_time'])['is_retweet'].apply(sum).reset_index().iloc[:, -1]

    posts['created_time'] = pd.to_datetime(posts['created_time'], format="%Y-%m-%d")
    posts['rank'] = posts.groupby(posts['ticker'])['created_time'].rank()
    return posts


def merge_days_posts(days, posts):
    posts_tmp = posts.copy().loc[:, ['ticker', 'created_time', 'rank', 'text']]
    for day in range(days):
        n = day + 1
        col = 'rank' + str(1)
        post_col = 'rank' + str(n) + 'posts'
        posts_tmp[col] = posts.apply(lambda x: int(x[3] + n), axis=1)
        posts_tmp = pd.merge(posts_tmp, posts.loc[:, ['ticker', 'rank', 'text']], how='left', left_on=['ticker', col],
                             right_on=['ticker', 'rank'], suffixes=['', '_r'])
        posts_tmp = posts_tmp.drop(['rank_r', col], axis=1)
        posts_tmp = posts_tmp.rename(columns={'text_r': 'text' + str(n)})
        posts_tmp = posts_tmp.dropna()
        posts_tmp['days_posts'] = posts_tmp.apply(lambda x: ' '.join(x[4:]), axis=1)
        final = posts_tmp.loc[:, ['ticker', 'created_time', 'days_posts']]
    return final


def tokenize(posts):
    tokenizer = RegexpTokenizer(r'\w+')
    posts["tokens"] = posts["days_posts"].apply(tokenizer.tokenize)
    posts = posts.loc[:, ['ticker', 'created_time', 'tokens']].reset_index().iloc[:, 1:]
    return posts


def get_close_price_nday(day):
    closeeps = pd.read_csv('closeeps.csv', low_memory=False).iloc[:, 2:]
    print(closeeps)
    closeeps = closeeps.drop(['cshoc', 'cshtrd', 'eps'], axis=1)
    closeeps.columns = ['created_time', 'ticker', 'cp']
    closeeps['created_time'] = pd.to_datetime(closeeps['created_time'], format="%Y%m%d")
    closeeps['ticker'] = '$' + closeeps['ticker']
    closeeps['rank'] = closeeps.groupby(closeeps['ticker'])['created_time'].rank()
    closeeps['rank2'] = closeeps.apply(lambda x: x['rank'] + day, axis=1)
    close = pd.merge(closeeps, closeeps, left_on=['ticker', 'rank2'], right_on=['ticker', 'rank'], suffixes=['', '_r'])
    close = close.loc[:, ['ticker', 'created_time', 'cp', 'cp_r']]
    close = close.rename(columns={'cp_r': 'cp' + str(day)})
    return close


def merge_posts_price(token_data, closeprice, target_rr):
    data = pd.merge(token_data, closeprice, on=['ticker', 'created_time'], suffixes=['', ''])
    data['rr'] = data.apply(lambda x: (x[-1] - x[-2]) / (x[-2]), axis=1)
    data['tf'] = data.apply(lambda x: x['rr'] >= target_rr, axis=1)
    return data


def word_to_vec(data):
    def get_average_word2vec(tokens_list, vector, generate_missing=False, k=300):
        if len(tokens_list) < 1:
            return np.zeros(k)
        if generate_missing:
            vectorized = [vector[word] if word in vector else np.random.rand(k) for word in tokens_list]
        else:
            vectorized = [vector[word] if word in vector else np.zeros(k) for word in tokens_list]

        length = len(vectorized)
        # print(length)
        summed = np.sum(vectorized, axis=0)
        averaged = np.divide(summed, length)
        return averaged

    def get_word2vec_embeddings(vectors, df, generate_missing=False):
        embeddings = df['tokens'].apply(lambda x: get_average_word2vec(x, vectors, generate_missing=generate_missing))
        return embeddings

    embeddings = get_word2vec_embeddings(word2vec, data).to_list()
    embeddings = np.array(embeddings)
    embeddings = pd.DataFrame(embeddings)
    #     pca = PCA(n_components=n_components, random_state=0)
    #     pcs = pd.DataFrame(pca.fit_transform(embeddings))
    #     tmp_data = pd.merge(data,embeddings, how='left', left_index=True, right_index=True, sort=True, copy=True,
    #                         indicator=False)
    #     tmp_data = tmp_data.drop(['tokens'], axis=1)
    #     tmp_data = tmp_data.dropna()
    #     tmp_data = tmp_data.reset_index().iloc[:, 1:]
    return embeddings


def get_post2_csv(sql):
    data = get_sql_data(sql)
    print('data read in' + str(len(data)))
    posts = all_posts_oneday(data)
    print('all_posts_oneday finish' + str(len(posts)))
    posts2 = merge_days_posts(5, posts)
    print('merge_posts finish' + str(len(posts2)))
    posts2.to_csv('posts2.csv')
    print('get posts2 csv file')


def get_w2v_csv():
    reader = pd.read_csv('posts2.csv', chunksize=10000)
    loop = True
    n = 1
    for chunk in reader:
        chunk = chunk.iloc[:, 1:]
        chunk['created_time'] = pd.to_datetime(chunk['created_time'], format="%Y-%m-%d")
        token_chunk = tokenize(chunk)
        print(len(token_chunk))
        dataset_chunk = merge_posts_price(token_chunk, closeprice, 0)
        print(len(dataset_chunk))
        final_chunk = word_to_vec(dataset_chunk)
        print(len(final_chunk))
        if n == 1:
            # print(len(final_chunk))
            final_chunk.to_csv('final_dataset.csv', mode='w')
            print(n)
            n += 1
        else:
            final_chunk.to_csv('final_dataset.csv', mode='a', header=None)
            print(n)
            n += 1


def main():
    path_to_model = 'googlew2v.bin'
    output_file = 'googlew2v.txt'
    bin2txt(path_to_model, output_file)


def bin2txt(path_to_model, output_file):
    output = codecs.open(output_file, 'w', 'utf-8')
    model = gensim.models.KeyedVectors.load_word2vec_format(path_to_model, binary=True)
    print('Done loading Word2Vec!')
    vocab = model.vocab
    for item in vocab:
        vector = list()
        for dimension in model[item]:
            vector.append(str(dimension))
        vector_str = ",".join(vector)
        line = item + "\t" + vector_str
        output.writelines(line + "\n")  # 本来用的是write（）方法，但是结果出来换行效果不对。改成writelines（）方法后还没试过。
    output.close()


def draw_3d(model):
    model = gensim.models.KeyedVectors.load_word2vec_format('googlew2v.bin', binary=True)
    print('pre-trained bin read in ')
    vocab = model.vocab
    with open('finance_words.txt', 'r', encoding='utf-8') as f:
        words = f.read()
    sample = words.split('\n')
    sample1 = []
    dic = list(vocab.keys())
    for i in range(0, len(sample)):
        if sample[i] in dic:
            sample1.append(sample[i])
    sample = sample1[:50]
    df_sample = pd.DataFrame(model[sample])
    df_sample.insert(0, 'word', sample)
    pca = PCA(n_components=3, random_state=0)
    pcs = pd.DataFrame(pca.fit_transform(df_sample.iloc[:, 1:]))
    pcs.insert(0, 'word', df_sample['word'])
    return pcs


if __name__ == '__main__':
    sql = 'select * from preprocessed_text_test'
    word2vec_path = "C:/Users/guoti/Desktop/group/googlew2v.bin"
    word2vec = gensim.models.KeyedVectors.load_word2vec_format(word2vec_path, binary=True)
    print('w2c bin file read in')
    closeprice = get_close_price_nday(3)
    print('cp file read in')
    get_post2_csv(sql)
    get_w2v_csv()
    pcs = draw_3d(word2vec)

    print('start to draw')
    fig = plt.figure()
    ax1 = plt.axes(projection='3d')
    z = pcs.iloc[:, 1]
    x = pcs.iloc[:, 2]
    y = pcs.iloc[:, 3]
    ax1.scatter3D(x, y, z)
    for i in range(0, len(x)):
        ax1.text(pcs.iloc[i, 1], pcs.iloc[i, 2], pcs.iloc[i, 3], pcs['word'][i])
    plt.show()
